In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score

In [44]:
dataframes = []
for i in range(1,6):
    df = pd.read_csv(f"../../../dataframes/double_gauss_{i}", index_col=0)
    df = df.loc[df["split"]=="test",["Z","Z_pred","Z_pred_std","Z_spec_prob","faint"]]
    dataframes.append(df)

In [45]:
mse = mean_squared_error
r2 = r2_score

In [46]:
test_mean_squared_errors = []
test_r2_scores = []
faint_mean_squared_errors = []
faint_r2_scores = []
test_probs = []
faint_probs = []
for df in dataframes:
    test_df = df.loc[~df["faint"]]
    faint_df = df.loc[df["faint"]]
    
    test_mean_squared_errors.append(mse(test_df["Z"], test_df["Z_pred"]))
    test_r2_scores.append(r2(test_df["Z"], test_df["Z_pred"]))
    faint_mean_squared_errors.append(mse(faint_df["Z"], faint_df["Z_pred"]))
    faint_r2_scores.append(r2(faint_df["Z"], faint_df["Z_pred"]))

    test_probs.append(test_df["Z_spec_prob"])
    faint_probs.append(faint_df["Z_spec_prob"])

test_mean_squared_error = np.array(test_mean_squared_errors).mean()
test_r2_score = np.array(test_r2_scores).mean()
faint_mean_squared_error = np.array(faint_mean_squared_errors).mean()
faint_r2_score = np.array(faint_r2_scores).mean()

In [52]:
np.array(faint_probs).mean(axis=0).min()

np.float64(3.802076639999999e-38)

In [42]:
epsilon = 1E-10
test_NLL, faint_NLL = -np.log(np.clip(np.array(test_probs).mean(axis=0), epsilon, None)).mean(), -np.log(np.clip(np.array(faint_probs).mean(axis=0), epsilon, None)).mean()

In [43]:
pd.DataFrame({"test":
                {"MSE": test_mean_squared_error,
                "R^2": test_r2_score,
                "NLL": test_NLL},
                "faint":
                {"MSE": faint_mean_squared_error,
                "R^2": faint_r2_score,
                "NLL": faint_NLL}
               }).T

,MSE,R^2,NLL
test,0.093072,0.833931,-0.653799
faint,0.218361,0.360328,0.243540


In [114]:
results = pd.DataFrame({"test":
                {"MSE": test_mean_squared_error,
                "R^2": test_r2_score,
                "NLL": test_NLL},
                "faint":
                {"MSE": faint_mean_squared_error,
                "R^2": faint_r2_score,
                "NLL": faint_NLL}
               }).T

# results.to_csv("../metrics/double_gauss")